In [ ]:
import pandas as pd
import re

df_low_freq_words = pd.read_csv("all_words_freq_sender.csv")
df_low_freq_words.head(2)

,word,freq
0,"group,",31
1,of,176


In [ ]:
df_low_freq_words = df_low_freq_words.sort_values(["freq"], ascending=[False])
df_low_freq_words.head(5)

,word,freq
48,management,422
49,claims,411
47,inc.,408
21,"services,",395
272,box,353


In [ ]:
low_weightage_words = list(df_low_freq_words[df_low_freq_words["freq"]>3]["word"])
len(low_weightage_words)

367

In [ ]:
def word2ngrams(text, n=3):
  """ Convert word into character ngrams. """
  return set([text[i:i+n] for i in range(len(text)-n+1)])

In [ ]:
def get_functional_metrics(gt,pred):

    #remove extra spaces in bw words
    gt = re.sub(' +', ' ', gt)
    pred = re.sub(' +', ' ', pred)
    
    ## remove low- weightaged words from both GT and pred
    gt_words = gt.lower().split(' ')
    pred_words = pred.lower().split(' ')

    norm_gt_words = []
    for word in gt_words:
        if word not in low_weightage_words:
            norm_gt_words.append(word)
    norm_gt = ' '.join(norm_gt_words)  

    norm_pred_words = []
    for word in pred_words:
        if word not in low_weightage_words:
            norm_pred_words.append(word)
    norm_pred = ' '.join(norm_pred_words)

    print("norm gt, norm pred : ", norm_gt,"\n", norm_pred)

    # get n-grams for both GT and Pred, and find the precision and recall bw them
    len_norm_gt, len_norm_pred = len(norm_gt), len(norm_pred)
    max_ngrams = (min(len_norm_gt,len_norm_pred)) # -1/2 ??
    weighted_f1_score_num = 0

    for i in range(2, max_ngrams+1):
        gt_igrams = word2ngrams(norm_gt, n=i)
        pred_igrams = word2ngrams(norm_pred, n=i)

        n_common_elements = len(gt_igrams) - (len(gt_igrams.difference(pred_igrams)))
        # print("n_common_elements : ", n_common_elements)
        precision = n_common_elements/len(pred_igrams)
        recall = n_common_elements/len(gt_igrams)

        # print(f"precision and recall at {i} is {precision} and {recall}")
        i_f1_score = 2*precision*recall / (precision+recall)
        weighted_f1_score_num += ((i/max_ngrams)*i_f1_score)
        # print("i ((i/max_ngrams)*fp) : ", i, ((i/max_ngrams)*i_f1_score), i_f1_score)

    weighted_f1_score_denom = (max_ngrams+1)/2 - (1/max_ngrams)
    print("weighted_f1_score_denom : ", weighted_f1_score_denom)
    return weighted_f1_score_num/weighted_f1_score_denom


        


In [ ]:
gt = "Edwin K. Stone Attorney at Law"
pred = "Edwin K."

get_functional_metrics(gt,pred)

gt pred :  ['edwin', 'k.', 'stone', 'attorney', 'at', 'law'] ['edwin', 'k.']
norm gt, norm pred :  edwin stone 
 edwin
n_common_elements :  4
precision and recall at 2 is 1.0 and 0.4
i ((i/max_ngrams)*fp) :  2 0.22857142857142862 0.5714285714285715
n_common_elements :  3
precision and recall at 3 is 1.0 and 0.3333333333333333
i ((i/max_ngrams)*fp) :  3 0.3 0.5
n_common_elements :  2
precision and recall at 4 is 1.0 and 0.25
i ((i/max_ngrams)*fp) :  4 0.32000000000000006 0.4
n_common_elements :  1
precision and recall at 5 is 1.0 and 0.14285714285714285
i ((i/max_ngrams)*fp) :  5 0.25 0.25
weighted_f1_score_denom :  2.8


0.3923469387755103